In [1]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

## Imports

In [2]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import datetime
from tqdm import tqdm
from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import pickle
import pandas
from pathlib import Path
import subprocess
from scipy.optimize import curve_fit
from scipy.stats import crystalball, chisquare
import sqlite3
from math import ceil
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from fnmatch import fnmatch
import hist
from hist import Hist

## Specify board name

In [3]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_name = "02D5_#12"
# chip_figname = f"LowBiasCurrent_HVoff_C26Removed_Ext10KOhm_{chip_name}"
# chip_figtitle= "LowBiasCurrent HVoff C26Removed Ext10KOhm "+chip_name
# CblHVandGNDExtDouble
# chip_figname = f"LowBiasCurrent_HVoff_L7Removed_50kFIFO_L1A_{chip_name}"
# chip_figtitle= "LowBiasCurrent HVoff L7Removed 50kFIFO L1A "+chip_name

chip_figname = f"LowBiasCurrent_HVoff_boardModJul18_AdditionalWireBonding_{chip_name}"
chip_figtitle= "LowBiasCurrent HVoff boardModJul18 AdditionalWireBonding "+chip_name

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

In [13]:
BL_map_THCal = np.zeros((16,16))
NW_map_THCal = np.zeros((16,16))

### Define Pixel For Plotting

In [17]:
DAC_row_list = [7]
DAC_col_list = [7]
DAC_scan_list = list(zip(DAC_col_list, DAC_row_list))

### Define DACs

In [18]:
#print(BL_map_THCal[row][col]-3*NW_map_THCal[row][col])

# min_threshold = int(BL_map_THCal[row][col]-3*NW_map_THCal[row][col])
min_threshold = 200
scan_step = 20

### Nominal chip
thresholds = {
    5: np.arange(min_threshold,  min_threshold + 60,  scan_step),
    6: np.arange(min_threshold,  min_threshold + 60,  scan_step),
    8: np.arange(min_threshold,  min_threshold + 90,  scan_step),
    10: np.arange(min_threshold, min_threshold + 100, scan_step),
    12: np.arange(min_threshold, min_threshold + 120, scan_step),
    15: np.arange(min_threshold, min_threshold + 130, scan_step),
    17: np.arange(min_threshold, min_threshold + 160, scan_step),
    20: np.arange(min_threshold, min_threshold + 180, scan_step),
    22: np.arange(min_threshold, min_threshold + 200, scan_step),
    25: np.arange(min_threshold, min_threshold + 220, scan_step),
    27: np.arange(min_threshold, min_threshold + 240, scan_step),
    30: np.arange(min_threshold, min_threshold + 260, scan_step),
}

### FFF corner wafer
# thresholds = {
#     # 5: np.arange(min_threshold,  min_threshold + 65,  scan_step),
#     5: np.arange(min_threshold,  min_threshold + 100,  scan_step),
#     6: np.arange(min_threshold,  min_threshold + 100,  scan_step),
#     8: np.arange(min_threshold,  min_threshold + 110,  scan_step),
#     # 10: np.arange(min_threshold, min_threshold + 80, scan_step),
#     10: np.arange(min_threshold, min_threshold + 120, scan_step),
#     12: np.arange(min_threshold, min_threshold + 130, scan_step),
#     15: np.arange(min_threshold, min_threshold + 140, scan_step),
#     17: np.arange(min_threshold, min_threshold + 150, scan_step),
#     # 20: np.arange(min_threshold, min_threshold + 100, scan_step),
#     20: np.arange(min_threshold, min_threshold + 160, scan_step),
#     22: np.arange(min_threshold, min_threshold + 170, scan_step),
#     # 25: np.arange(min_threshold, min_threshold + 65, scan_step),
#     25: np.arange(min_threshold, min_threshold + 180, scan_step),
#     27: np.arange(min_threshold, min_threshold + 190, scan_step),
#     30: np.arange(min_threshold, min_threshold + 200, scan_step),
# }


# thresholds = np.arange(233,450,2)
# thresholds = np.arange(451,500,2)
# thresholds = [10, 150, 240, 250, 260, 270, 280, 290, 300]

### Define Charges

In [19]:
# Full Charges
# QInjEns = [5, 6, 8, 10, 12, 15, 17, 20, 22, 25, 27, 30]
# Recommend for TID
# QInjEns = [5, 12, 17, 22, 27]
# Single
QInjEns = [15]
ACC_map_full_Scurve = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}

## Counts Plotting

In [ ]:
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
hitmap_full_Scurve = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
sum_data_hitmap_full_Scurve = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
sum2_data_hitmap_full_Scurve = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
for index, row, col in zip((range(len(DAC_row_list))), DAC_row_list, DAC_col_list):
    for QInj in (QInjEns):
        for DAC in (thresholds[QInj]):
            print(f'Pixel {col},{row} - {QInj} fC - DAC {DAC}')
            path_pattern = f"*{today.isoformat()}_Array_Test_Results/E2_testing_VRef_SCurve_Pixel_C{col}_R{row}_QInj_{QInj}_Threshold_{DAC}_HVoff_pf_hits"
            # path_pattern = f"*2023-07-24_Array_Test_Results/E2_testing_VRef_SCurve_Pixel_C{col}_R{row}_QInj_{QInj}_Threshold_{DAC}_HVoff_pf_hits"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
                        print(file_list[-1])
            total_files = len(file_list)
            for file_index, file_name in enumerate(file_list):
                print(f"{file_index+1}/{total_files}")
                with open(file_name) as infile:
                    data = False
                    first_line = infile.readline()
                    FPGA_state = first_line.split(',')[0]
                    old_FPGA_state = FPGA_state
                    FPGA_data = 0
                    FPGA_data2 = 0
                    for line in infile:
                        text_list = line.split(',')
                        FPGA_state = text_list[0]
                        FPGA_data = int(text_list[3])
                        FPGA_data2 = int(text_list[3])**2
                        if(FPGA_state=='0'): continue
                        if(FPGA_state=='1'): continue
                        if FPGA_state != old_FPGA_state:
                            sum_data_hitmap_full_Scurve[row][col][QInj][DAC] += FPGA_data
                            sum2_data_hitmap_full_Scurve[row][col][QInj][DAC] += FPGA_data2
                            hitmap_full_Scurve[row][col][QInj][DAC] += 1
                            FPGA_data = 0
                            FPGA_data2 = 0
                            old_FPGA_state = FPGA_state

In [ ]:
data_mean = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
data_std = {row:{col:{q:{thr:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
for index, row, col in zip((range(len(DAC_row_list))), DAC_row_list, DAC_col_list):
    for QInj in (QInjEns):
        for DAC in (thresholds[QInj]):
            if(hitmap_full_Scurve[row][col][QInj][DAC]==0): 
                data_mean[row][col][QInj][DAC] = 0
                data_std[row][col][QInj][DAC] = 0
                continue
            data_mean[row][col][QInj][DAC] = sum_data_hitmap_full_Scurve[row][col][QInj][DAC]/hitmap_full_Scurve[row][col][QInj][DAC]
            data_std[row][col][QInj][DAC] = np.sqrt((sum2_data_hitmap_full_Scurve[row][col][QInj][DAC]/hitmap_full_Scurve[row][col][QInj][DAC]) - pow(data_mean[row][col][QInj][DAC], 2))

In [ ]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
# colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

fig = plt.figure(dpi=200, figsize=(8,4.5))
# fig = plt.figure(dpi=200, figsize=(15,30))
gs = fig.add_gridspec(len(DAC_scan_list),len(DAC_scan_list))
u_cl = np.sort(np.unique(DAC_col_list))
u_rl = np.sort(np.unique(DAC_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='k', ls='--', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='k', ls='--', lw=0.7)
        # ax0.axvline(GMean_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        # ax0.axvline(GMean_map_THCal[row][col]+2*GSigma_map_THCal[row][col], color='k', ls='--', label="THCal NW", lw=0.7)
        # ax0.axvline(GMean_map_THCal[row][col]-2*GSigma_map_THCal[row][col], color='k', ls='--', lw=0.7)
        for i, QInj in enumerate(QInjEns[0:]):
            # ax0.plot(thresholds[QInj], hitmap_full_Scurve[row][col][QInj].values(), '.-', label=f"{QInj} fC",lw=0.5,markersize=2)
            ax0.plot(thresholds[QInj], data_mean[row][col][QInj].values(), '.-', color=colors[i], label=f"{QInj} fC",lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="center right")
        plt.yscale("log")
    plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Full S-Curve")
    plt.tight_layout()
plt.savefig(fig_path+"/Full_S-Curve_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

#### Additional plotting

In [ ]:
def find_largest_non_zero_idx(array, key_array):
    array = np.array(list(array))
    np_key = np.where(array!=0, key_array, 0)
    return key_array[np.argmax(np_key)]

In [ ]:
fig = plt.figure(dpi=200, figsize=(8,4.5))
gs = fig.add_gridspec(len(DAC_scan_list),len(DAC_scan_list))
u_cl = np.sort(np.unique(DAC_col_list))
u_rl = np.sort(np.unique(DAC_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axhline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axhline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='k', ls='--', label="THCal NW", lw=0.7)
        ax0.axhline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='k', ls='--', lw=0.7)
        X = []
        Y = []
        for QInj in QInjEns[0:]:
            tp_thr = find_largest_non_zero_idx(hitmap_full_Scurve[row][col][QInj].values(), thresholds[QInj])
            X.append(QInj)
            Y.append(tp_thr)
        ax0.plot(X, Y,'ro')
        m, b = np.polyfit(X, Y, 1)
        topEdge = BL_map_THCal[row][col]+NW_map_THCal[row][col]
        min_charge = (topEdge - b)/m
        ax0.plot(np.linspace(min_charge,30,100), b + m*np.linspace(min_charge,30,100), 'b-', label=f'DAC = {m:.3f}Q + {b:.3f}')
        ax0.set_xlabel("Charge Injected (Allegedly) [fC]")
        ax0.set_ylabel("DAC Threshold [decimal]")
        plt.legend(loc="center right")
plt.title(f"{chip_name}, Pixel ({row},{col}) Full S-Curve, PA Low Bias Current Mode")
plt.tight_layout()
plt.show()

In [ ]:
row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")
# Loop for threshold calibration
# for row in [3]:
#     for col in [3]:
for index,row,col in zip(tqdm(len(DAC_row_list))), DAC_row_list, DAC_col_list):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # Maybe required to make this work
        # pixel_decoded_register_write("enable_TDC", "0")
        # pixel_decoded_register_write("testMode_TDC", "0")
        # Enable THCal clock and buffer, disable bypass
        pixel_decoded_register_write("CLKEn_THCal", "1")
        pixel_decoded_register_write("BufEn_THCal", "1")
        pixel_decoded_register_write("Bypass_THCal", "0")
        pixel_decoded_register_write("TH_offset", format(0x07, '06b'))
        # Reset the calibration block (active low)
        pixel_decoded_register_write("RSTn_THCal", "0")
        pixel_decoded_register_write("RSTn_THCal", "1")
        # Start and Stop the calibration, (25ns x 2**15 ~ 800 us, ACCumulator max is 2**15)
        pixel_decoded_register_write("ScanStart_THCal", "1")
        pixel_decoded_register_write("ScanStart_THCal", "0")
        # Check the calibration done correctly
        if(pixel_decoded_register_read("ScanDone", "Status")!="1"): print("!!!ERROR!!! Scan not done!!!")
        BL_map_THCal[row, col] = pixel_decoded_register_read("BL", "Status", need_int=True)
        NW_map_THCal[row, col] = pixel_decoded_register_read("NW", "Status", need_int=True)
        # Disable clock and buffer before charge injection 
        pixel_decoded_register_write("CLKEn_THCal", "0") 
        pixel_decoded_register_write("BufEn_THCal", "0")
        # Set Charge Inj Q to 15 fC
        pixel_decoded_register_write("QSel", format(0x0e, '05b'))

## Define Pixels for DAQ scan to study multiple pixels TOT and TOA

In [ ]:
# Diagnoal scan
# col_list = list(np.arange(16)[1:-1]) + list(np.arange(16)[1:-1])
# row_list = list(np.arange(16)[1:-1]) + list(np.arange(16)[::-1][1:-1])

# Box scan
# col_list = list(np.arange(16)[0:-1]) + list(np.full(15, 15)) + list(np.arange(16)[1:]) + list(np.full(15, 0))
# row_list = list(np.full(15, 0)) + list(np.arange(16)[0:-1]) + list(np.full(15, 15)) + list(np.arange(16)[1:])

# Cluster scan
# col_list = [3, 3, 3, 12, 12, 12]
# row_list = [3, 8, 13, 3, 8, 13]

#row_list = list(np.arange(4)[0:])
#col_list = np.zeros_like(row_list)
# row_list = [3,12,3,12]
# col_list = [3,3,12,12]
# row_list = [14, 14, 14, 14]
# col_list = [6, 7, 8, 9]
# row_list = [3,12]
# col_list = [3,3]
#row_list = [3]
#col_list = [3]
# print(col_list)
# print(row_list)

# Sensor board pixels of interest
# row_list = [15, 2, 7, 11]
# col_list = [6, 3, 6, 12]

# all pixels
row_list = list(np.arange(16)[0:])
col_list = list(np.arange(16)[0:])
scan_list = []
for r in row_list:
    for c in col_list:
        scan_list.append((c,r))

# row_list = [15]
# col_list = [6]

# Four sensor and one non sensor
# row_list = [14, 14, 14, 14, 15]
# col_list = [6, 7, 8, 9, 6]
# scan_list = list(zip(col_list, row_list))

print(row_list)
print(col_list)
print(len(scan_list))
print(scan_list)